In [12]:
# imports
import pandas as pd
import numpy as np
import datetime
import re

### Load data

In [6]:
RATINGS_FILE = 'ml-latest-small/ratings.csv'
TAGS_FILE = 'ml-latest-small/tags.csv'
MOVIES_FILE = 'ml-latest-small/movies.csv'

#### Load Data
print("Loading data...")
try:
    df_ratings = pd.read_csv(RATINGS_FILE)
    df_tags = pd.read_csv(TAGS_FILE)
    df_movies = pd.read_csv(MOVIES_FILE)
    print("Data loaded successfully.")
except FileNotFoundError as e:
    print(f"Error loading files: {e}")
    print("Please ensure the file paths in the script are correct.")
    exit() # Exit if files aren't found
except Exception as e:
    print(f"An error occurred during file loading: {e}")

Loading data...
Data loaded successfully.


### Timestamps

In [7]:
print("\n--- 1. Timestamps (ratings.csv, tags.csv) ---")
# --- Check ---
print("Checking Timestamps in ratings.csv:")
print(f"  Data type: {df_ratings['timestamp'].dtype}")
min_ts_rating = df_ratings['timestamp'].min()
max_ts_rating = df_ratings['timestamp'].max()
print(f"  Min timestamp: {min_ts_rating} -> {datetime.datetime.utcfromtimestamp(min_ts_rating)}")
print(f"  Max timestamp: {max_ts_rating} -> {datetime.datetime.utcfromtimestamp(max_ts_rating)}")

print("\nChecking Timestamps in tags.csv:")
print(f"  Data type: {df_tags['timestamp'].dtype}")
min_ts_tag = df_tags['timestamp'].min()
max_ts_tag = df_tags['timestamp'].max()
print(f"  Min timestamp: {min_ts_tag} -> {datetime.datetime.utcfromtimestamp(min_ts_tag)}")
print(f"  Max timestamp: {max_ts_tag} -> {datetime.datetime.utcfromtimestamp(max_ts_tag)}")
print("  Check result: Timestamps are integers (Unix epoch time). Format looks consistent.")


--- 1. Timestamps (ratings.csv, tags.csv) ---
Checking Timestamps in ratings.csv:
  Data type: int64
  Min timestamp: 828124615 -> 1996-03-29 18:36:55
  Max timestamp: 1537799250 -> 2018-09-24 14:27:30

Checking Timestamps in tags.csv:
  Data type: int64
  Min timestamp: 1137179352 -> 2006-01-13 19:09:12
  Max timestamp: 1537098603 -> 2018-09-16 11:50:03
  Check result: Timestamps are integers (Unix epoch time). Format looks consistent.


### Ratings

In [8]:
print("\n--- 2. Ratings Scale (ratings.csv) ---")
# --- Check ---
print("Checking Ratings Scale:")
unique_ratings = sorted(df_ratings['rating'].unique())
min_rating = df_ratings['rating'].min()
max_rating = df_ratings['rating'].max()
print(f"  Unique rating values: {unique_ratings}")
print(f"  Min rating: {min_rating}")
print(f"  Max rating: {max_rating}")
# Check for ratings outside the expected 0.5-5.0 range
invalid_ratings = df_ratings[ (df_ratings['rating'] < 0.5) | (df_ratings['rating'] > 5.0) ]
print(f"  Number of ratings outside expected range (0.5-5.0): {len(invalid_ratings)}")
print("  Check result: Ratings appear to be within the documented 0.5-5.0 range with 0.5 increments.")



--- 2. Ratings Scale (ratings.csv) ---
Checking Ratings Scale:
  Unique rating values: [0.5, 1.0, 1.5, 2.0, 2.5, 3.0, 3.5, 4.0, 4.5, 5.0]
  Min rating: 0.5
  Max rating: 5.0
  Number of ratings outside expected range (0.5-5.0): 0
  Check result: Ratings appear to be within the documented 0.5-5.0 range with 0.5 increments.


### Tags

In [9]:
print("\n--- 3. Tags (tags.csv) ---")
# --- Check ---
print("Checking Tags:")
nan_tags_count = df_tags['tag'].isnull().sum()
print(f"  Number of NaN/Null tags: {nan_tags_count}")
print("  Example tags (first 5 non-null):")
print(df_tags['tag'].dropna().head().tolist())
# Check for leading/trailing whitespace
tags_with_whitespace = df_tags[df_tags['tag'].ne(df_tags['tag'].str.strip()) & df_tags['tag'].notnull()]
print(f"  Number of tags with leading/trailing whitespace: {len(tags_with_whitespace)}")
# Check case differences (example: 'funny' vs 'Funny')
if nan_tags_count == 0: # Avoid error if all tags are NaN
    tag_counts_case_sensitive = df_tags['tag'].value_counts()
    tag_counts_case_insensitive = df_tags['tag'].str.lower().value_counts()
    print(f"  Unique tags (case sensitive): {len(tag_counts_case_sensitive)}")
    print(f"  Unique tags (case insensitive): {len(tag_counts_case_insensitive)}")
    if len(tag_counts_case_sensitive) != len(tag_counts_case_insensitive):
        print("  Check result: Case differences detected. Whitespace or NaN may exist.")
    else:
         print("  Check result: Case seems consistent, but check whitespace/NaN.")
else:
     print("  Check result: NaN tags found.")


--- 3. Tags (tags.csv) ---
Checking Tags:
  Number of NaN/Null tags: 0
  Example tags (first 5 non-null):
['funny', 'Highly quotable', 'will ferrell', 'Boxing story', 'MMA']
  Number of tags with leading/trailing whitespace: 0
  Unique tags (case sensitive): 1589
  Unique tags (case insensitive): 1475
  Check result: Case differences detected. Whitespace or NaN may exist.


In [16]:
# Convert to lowercase and strip leading/trailing whitespace
#
#
#
# Important step to apply when using tags.csv
#
#
#
df_tags['tag_cleaned'] = df_tags['tag'].str.lower().str.strip()
print("  Converted tags to lowercase and stripped whitespace.")
print("  Example cleaned tags (first 5):")
print(df_tags[['tag', 'tag_cleaned']].head().values.tolist())

  Converted tags to lowercase and stripped whitespace.
  Example cleaned tags (first 5):
[['funny', 'funny'], ['Highly quotable', 'highly quotable'], ['will ferrell', 'will ferrell'], ['Boxing story', 'boxing story'], ['MMA', 'mma']]


### Genres

In [10]:
print("\n--- 4. Genres (movies.csv) ---")
# --- Check ---
print("Checking Genres:")
nan_genres_count = df_movies['genres'].isnull().sum()
print(f"  Number of NaN/Null genres: {nan_genres_count}")
print(f"  Unique genre strings (first 10): {df_movies['genres'].unique()[:10]}")
no_genres_listed_count = df_movies[df_movies['genres'] == '(no genres listed)'].shape[0]
print(f"  Count of movies with '(no genres listed)': {no_genres_listed_count}")
print("  Example genre string split:")
example_genre_string = df_movies['genres'].iloc[0]
print(f"    Original: '{example_genre_string}' -> Split: {example_genre_string.split('|')}")
print("  Check result: Genres are pipe-separated strings. Special value '(no genres listed)' exists.")


--- 4. Genres (movies.csv) ---
Checking Genres:
  Number of NaN/Null genres: 0
  Unique genre strings (first 10): ['Adventure|Animation|Children|Comedy|Fantasy'
 'Adventure|Children|Fantasy' 'Comedy|Romance' 'Comedy|Drama|Romance'
 'Comedy' 'Action|Crime|Thriller' 'Adventure|Children' 'Action'
 'Action|Adventure|Thriller' 'Comedy|Horror']
  Count of movies with '(no genres listed)': 34
  Example genre string split:
    Original: 'Adventure|Animation|Children|Comedy|Fantasy' -> Split: ['Adventure', 'Animation', 'Children', 'Comedy', 'Fantasy']
  Check result: Genres are pipe-separated strings. Special value '(no genres listed)' exists.


### Titles

In [13]:
print("\n--- 5. Titles (movies.csv) ---")
# --- Check ---
print("Checking Titles:")
nan_titles_count = df_movies['title'].isnull().sum()
print(f"  Number of NaN/Null titles: {nan_titles_count}")
# Regex to roughly check "Title (YYYY)" format. Allows variations.
# This regex looks for: Anything, space, literal '(', 4 digits, literal ')', end of string.
title_year_pattern = re.compile(r'.+ \(\d{4}\)$')
malformed_titles = df_movies[~df_movies['title'].str.match(title_year_pattern, na=False)]
print(f"  Number of titles potentially not ending in ' (YYYY)': {len(malformed_titles)}")
if not malformed_titles.empty:
    print("  Examples of potentially malformed titles (first 5):")
    print(malformed_titles['title'].head().tolist())
print("  Check result: Most titles seem to follow the format, but inconsistencies might exist (as per README).")


--- 5. Titles (movies.csv) ---
Checking Titles:
  Number of NaN/Null titles: 0
  Number of titles potentially not ending in ' (YYYY)': 24
  Examples of potentially malformed titles (first 5):
['From Dusk Till Dawn 2: Texas Blood Money (1999) ', 'Babylon 5', 'Justice League: The New Frontier (2008) ', 'Assembly (Ji jie hao) (2007) ', '96 Minutes (2011) ']
  Check result: Most titles seem to follow the format, but inconsistencies might exist (as per README).


#### General

In [14]:
print("\n--- 6. General Missing Values & Types ---")
# --- Check ---
print("Checking general info and missing values:")
print("\ndf_ratings info:")
df_ratings.info()
print("\ndf_ratings missing values:")
print(df_ratings.isnull().sum())

print("\ndf_tags info:")
df_tags.info()
print("\ndf_tags missing values:")
print(df_tags.isnull().sum()) # We saw 'tag' can have NaNs

print("\ndf_movies info:")
df_movies.info()
print("\ndf_movies missing values:")
print(df_movies.isnull().sum()) # We saw 'year' extraction can yield NaNs


--- 6. General Missing Values & Types ---
Checking general info and missing values:

df_ratings info:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100836 entries, 0 to 100835
Data columns (total 4 columns):
 #   Column     Non-Null Count   Dtype  
---  ------     --------------   -----  
 0   userId     100836 non-null  int64  
 1   movieId    100836 non-null  int64  
 2   rating     100836 non-null  float64
 3   timestamp  100836 non-null  int64  
dtypes: float64(1), int64(3)
memory usage: 3.1 MB

df_ratings missing values:
userId       0
movieId      0
rating       0
timestamp    0
dtype: int64

df_tags info:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3683 entries, 0 to 3682
Data columns (total 4 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   userId     3683 non-null   int64 
 1   movieId    3683 non-null   int64 
 2   tag        3683 non-null   object
 3   timestamp  3683 non-null   int64 
dtypes: int64(3), object(1)
memory 